In [20]:
#Import Libraries
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [21]:
# Autheticate E-Mail ID
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [22]:
# Find ID of character stat csv
filelist = drive.ListFile({'q': "title contains 'character_stats.csv' and trashed=false"}).GetList()

if len(filelist) > 1:
  raise SystemExit("ERROR: Multiple character_stats.csv files found!")
elif len(filelist) == 0:
  raise SystemExit("ERROR: No character_stats.csv files found!")
for file in filelist:
    print(file['title'], file['id'])
    char_stat_csv = file['id']

downloaded = drive.CreateFile({'id':char_stat_csv}) # replace the id with id of file you want to access
downloaded.GetContentFile('character_stats.csv')

character_stats.csv 13-5DHPtBQO5EjAuIYbigXLkrrkx_6DWv


In [23]:
# Import pandas package
import pandas as pd

# Read in character sheets and convert to dataframe
char_stat = pd.read_csv('character_stats.csv', index_col=0)
print(char_stat)

# Convert dataframe to dictionary
char_stat = char_stat.to_dict()
print(char_stat)

          LVL  STR  DEX  ...  Stealth  Survival             Professions
Kyli Ren    1   15   12  ...        0         4                     NaN
Mivyral     1   10   10  ...        5         1              Acting - 0
Orion       1   11   13  ...        1         6                     NaN
Pabu        1   10   15  ...        2         0  Maintenance Worker - 5

[4 rows x 40 columns]
{'LVL': {'Kyli Ren': 1, 'Mivyral': 1, 'Orion': 1, 'Pabu': 1}, 'STR': {'Kyli Ren': 15, 'Mivyral': 10, 'Orion': 11, 'Pabu': 10}, 'DEX': {'Kyli Ren': 12, 'Mivyral': 10, 'Orion': 13, 'Pabu': 15}, 'CON': {'Kyli Ren': 14, 'Mivyral': 8, 'Orion': 10, 'Pabu': 10}, 'INT': {'Kyli Ren': 10, 'Mivyral': 12, 'Orion': 16, 'Pabu': 12}, 'WIS': {'Kyli Ren': 11, 'Mivyral': 10, 'Orion': 15, 'Pabu': 10}, 'CHA': {'Kyli Ren': 11, 'Mivyral': 13, 'Orion': 4, 'Pabu': 16}, 'INIT': {'Kyli Ren': 1, 'Mivyral': 1, 'Orion': 1, 'Pabu': 2}, 'SP': {'Kyli Ren': 9, 'Mivyral': 5, 'Orion': 6, 'Pabu': 6}, 'HP': {'Kyli Ren': 11, 'Mivyral': 10, 'Orion'

In [24]:
# How to print nested dictionary value
print(char_stat['Weapons']['Pabu'])

{'Knife, Survival' : '1d4 S',  'Azimuth Lazer Pistol' : '1d4 F', 'Frag Grenade' : '1d6 P', 'Stickybomb Grenade' : '+2 AB'}


In [25]:
# Lists to convert from ability score to modifier
ability_val = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]
mod_val = [-5,-4,-4,-3,-3,-2,-2,-1,-1,0,0,1,1,2,2,3,3,4,4,5,5,6,6,7,7,8]

In [26]:
# Function for initialize checks
def initialize(character, roll, bonus=0):
    try:
        char_stat["DEX"][character]
    except KeyError:
        print(character,"is not a valid character")
    for i,val in enumerate(ability_val):
        if val == char_stat["DEX"][character]:
            dex_mod = mod_val[i]
            init_check = roll + dex_mod + bonus
    print("Initialize check =",roll,"(roll) +",dex_mod,"(DEXMOD) +",bonus,"(bonus) =",init_check)

In [27]:
initialize("Kyli Ren", 10)

Initialize check = 10 (roll) + 1 (DEXMOD) + 0 (bonus) = 11


In [28]:
# Function for combat results
def combat(combat_type, attacker, a_roll, defender, d_roll, a_bonus=0, d_bonus=0):
    try:
        char_stat[combat_type]
    except KeyError:
        print(combat_type,"is not an attack type")
        return
    try:
        attack_val = a_roll+char_stat[combat_type][attacker]+a_bonus
    except KeyError:
        print(attacker,"is not a valid attacker")
        return
    try:
        defend_val = char_stat['KAC'][defender]
    except KeyError:
        print(defender,"is not a valid defender")   
        return
    
    print("----------------------")
    print(attacker,"vs.",defender,"\n")
    damage = attack_val - defend_val
    print("Damage check =",attack_val,"(attack) -",defend_val, "(AC) =",damage,"\n")
    
    if damage > 0:
        print("\nAttack successful!")
        char_stat['SP'][defender] = char_stat['SP'][defender] - damage
        print(defender,"takes",damage,"damage...")
        if char_stat['SP'][defender] <= 0:
            char_stat['HP'][defender] = char_stat['HP'][defender] + char_stat['SP'][defender]
            print(defender,"stamina was broken!")
            if char_stat['HP'][defender] <= 0:
                char_stat['RP'][defender] = char_stat['RP'][defender] - 1
                print(defender,"has taken too much damage! They are on the verge of death!")
                if char_stat['RP'][defender] <= 0:
                    print("I'm sorry but",defender,"has died in battle!")
                else:                    
                    print(char_stat['RP'][defender],"RP left")            
            else:                    
                print(char_stat['HP'][defender],"HP left")    
        else:                    
            print(char_stat['SP'][defender],"SP left")
    else:
        print("\nAttack failed!")
        
    print("----------------------")
    return
        
    

In [29]:
attack_roll = 20
defend_roll = 4
combat("Melee","Kyli Ren",attack_roll,"Mivyral",defend_roll)
combat("Melee","Kyli Ren",attack_roll,"Orion",defend_roll)
combat("Melee","Kyli Ren",attack_roll,"Pabu",defend_roll)
combat("Melee","Kyli Ren",attack_roll,"Pabu",defend_roll)
combat("Melee","Kyli Ren",attack_roll,"Pabu",defend_roll)

----------------------
Kyli Ren vs. Mivyral 

Damage check = 24 (attack) - 12 (AC) = 12 


Attack successful!
Mivyral takes 12 damage...
Mivyral stamina was broken!
3 HP left
----------------------
----------------------
Kyli Ren vs. Orion 

Damage check = 24 (attack) - 13 (AC) = 11 


Attack successful!
Orion takes 11 damage...
Orion stamina was broken!
5 HP left
----------------------
----------------------
Kyli Ren vs. Pabu 

Damage check = 24 (attack) - 14 (AC) = 10 


Attack successful!
Pabu takes 10 damage...
Pabu stamina was broken!
4 HP left
----------------------
----------------------
Kyli Ren vs. Pabu 

Damage check = 24 (attack) - 14 (AC) = 10 


Attack successful!
Pabu takes 10 damage...
Pabu stamina was broken!
Pabu has taken too much damage! They are on the verge of death!
1 RP left
----------------------
----------------------
Kyli Ren vs. Pabu 

Damage check = 24 (attack) - 14 (AC) = 10 


Attack successful!
Pabu takes 10 damage...
Pabu stamina was broken!
Pabu has tak